Now that you have your Azure resources, an index, and model in place, you can run the script to chat with the index.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  
AZURE_SEARCH_SERVICE = os.getenv("AISEARCH_API_ENDPOINT_2")
AZURE_OPENAI_ACCOUNT = os.getenv("AZURE_OPENAI_ACCOUNT")
AZURE_DEPLOYMENT_MODEL = os.getenv("DEPLOYMENT_NAME")

In [2]:
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI


credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
openai_client = AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ACCOUNT,
    azure_ad_token_provider=token_provider
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE,
    index_name="hotels-sample-index",
    credential=credential
)

GROUNDED_PROMPT="""
You are a friendly assistant that recommends hotels based on activities and amenities.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

query="Can you recommend a few hotels with complimentary breakfast?"

search_results = search_client.search(
    search_text=query,
    top=5,
    select="Description,HotelName,Tags",
    query_type="semantic",
    semantic_configuration_name="semantic-config"
)
sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Tags"]}' for document in search_results])
print(sources_formatted)

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

print(response.choices[0].message.content)

Head Wind Resort:The best of old town hospitality combined with views of the river and cool breezes off the prairie. Our penthouse suites offer views for miles and the rooftop plaza is open to all guests from sunset to 10 p.m. Enjoy a complimentary continental breakfast in the lobby, and free Wi-Fi throughout the hotel.:['coffee in lobby', 'free wifi', 'view']
Waterfront Scottish Inn:Newly Redesigned Rooms & airport shuttle. Minutes from the airport, enjoy lakeside amenities, a resort-style pool & stylish new guestrooms with Internet TVs.:['24-hour front desk service', 'continental breakfast', 'free wifi']
Swan Bird Lake Inn:We serve a continental-style breakfast each morning, featuring a variety of food and drinks. Our locally made, oh-so-soft, caramel cinnamon rolls are a favorite with our guests. Other breakfast items include coffee, orange juice, milk, cereal, instant oatmeal, bagels, and muffins.:['continental breakfast', 'free wifi', '24-hour front desk service']
Thunderbird Mote

In [3]:
import json

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a few hotels that offer complimentary breakfast? Tell me their description, address, tags, and the rate for one room they have which sleep 4 people."

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
selected_fields = ["HotelName","Description","Address","Rooms","Tags"]
search_results = search_client.search(
    search_text=query,
    top=5,
    select=selected_fields,
    query_type="semantic",
    semantic_configuration_name="semantic-config"

)
sources_filtered = [{field: result[field] for field in selected_fields} for result in search_results]
sources_formatted = "\n".join([json.dumps(source) for source in sources_filtered])
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

print(response.choices[0].message.content)


Here are some great hotel options that offer complimentary breakfast and have rooms that sleep 4 people:

1. **Head Wind Resort**
   - **Description:** Combines old town hospitality with river views and prairie breezes. Features a rooftop plaza open evenings and complimentary continental breakfast in the lobby. Free Wi-Fi throughout.
   - **Address:** 7633 E 63rd Pl, Tulsa, OK, 74133, USA.
   - **Room:** "Suite, 2 Queen Beds (Amenities)" with 2 Queen Beds, sleeps 4, at $254.99 per night. Non-smoking.
   - **Tags:** Coffee maker.

2. **Nordick's Valley Motel**
   - **Description:** Close to the nation's capital, offering access to hiking, wine tasting, and caverns. Includes specially priced packages and continental breakfast.
   - **Address:** 1401 I St NW, Washington D.C., 20005, USA.
   - **Room:** "Standard Room, 2 Queen Beds (City View)" with 2 Queen Beds, sleeps 4, at $135.99 per night. Smoking allowed.
   - **Tags:** Suite, coffee maker.

3. **Waterfront Scottish Inn**
   - **Desc